In [38]:
import pandas as pd
import numpy as np
import seaborn as sns; sns.set('talk')
import matplotlib.pyplot as plt
params = {'legend.fontsize': 'large',
          'figure.figsize': (16, 9),
         'axes.labelsize': 'medium',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'small',
         'ytick.labelsize':'small'}
plt.rcParams.update(params)
import json
from pandas.io.json import json_normalize

In [55]:
with open('edge intelligence.json') as json_file:
    data = json.load(json_file)

In [56]:
df = pd.DataFrame()
df = json_normalize(data,)

In [59]:
df.loc[(df['correct']==False) & (df['index_top5'] != -1)]

,sample,exit,prediction,scores,target,time,time_tx,correct,index_top5
37,9,1,"[92, 13, 77, 0, 36]","[0.5218485593795776, 0.2361373007297516, 0.123...",0,302.4950,4.1626,False,3
38,9,2,"[92, 0, 36, 13, 65]","[0.7374869585037231, 0.1854843944311142, 0.057...",0,365.6264,4.1626,False,1
44,11,0,"[13, 0, 77, 36, 92]","[0.4231765568256378, 0.291526734828949, 0.0912...",0,210.9162,4.8321,False,1
68,17,0,"[92, 0, 65, 36, 70]","[0.910774827003479, 0.06875967234373093, 0.007...",0,217.3178,4.5211,False,1
73,18,1,"[54, 83, 0, 23, 62]","[0.725153923034668, 0.12804390490055084, 0.099...",0,308.4937,4.2576,False,2
...,...,...,...,...,...,...,...,...,...
390,97,2,"[74, 67, 1, 33, 49]","[0.6636925935745239, 0.31221604347229004, 0.01...",1,400.0647,8.0242,False,2
391,97,3,"[74, 67, 33, 1, 99]","[0.4515804350376129, 0.2798946797847748, 0.258...",1,427.9529,8.0242,False,3
393,98,1,"[94, 63, 1, 99, 72]","[0.36964479088783264, 0.15177731215953827, 0.1...",1,320.7654,4.8565,False,2
394,98,2,"[94, 63, 1, 43, 72]","[0.6176510453224182, 0.3412763178348541, 0.019...",1,400.7997,4.8565,False,2


In [60]:
df[36:40]

,sample,exit,prediction,scores,target,time,time_tx,correct,index_top5
36,9,0,"[36, 97, 92, 77, 13]","[0.24251487851142883, 0.16510248184204102, 0.1...",0,206.9919,4.1626,False,-1
37,9,1,"[92, 13, 77, 0, 36]","[0.5218485593795776, 0.2361373007297516, 0.123...",0,302.4950,4.1626,False,3
38,9,2,"[92, 0, 36, 13, 65]","[0.7374869585037231, 0.1854843944311142, 0.057...",0,365.6264,4.1626,False,1
39,9,3,"[0, 92, 65, 13, 97]","[0.9971765279769897, 0.0010480874916538596, 0....",0,390.2028,4.1626,True,0


In [87]:
for i, d in df.groupby(['sample', 'exit']):
    if d['correct'].any()==False:
        print('{} {} {}'.format(str(d['sample'].item()), str(d['exit'].item()), d['prediction'].item())) 

9 0 [36, 97, 92, 77, 13]
9 1 [92, 13, 77, 0, 36]
9 2 [92, 0, 36, 13, 65]
11 0 [13, 0, 77, 36, 92]
17 0 [92, 0, 65, 36, 70]
18 1 [54, 83, 0, 23, 62]
19 0 [44, 60, 75, 95, 42]
19 1 [60, 0, 92, 13, 65]
21 2 [7, 92, 0, 87, 97]
23 0 [65, 0, 39, 97, 87]
26 0 [77, 97, 66, 92, 70]
26 1 [87, 92, 77, 97, 13]
26 2 [92, 0, 97, 87, 77]
27 0 [92, 95, 29, 42, 0]
30 0 [59, 11, 56, 4, 28]
30 1 [59, 0, 98, 29, 2]
30 2 [97, 59, 87, 0, 77]
30 3 [59, 0, 97, 87, 19]
36 0 [36, 0, 68, 92, 5]
36 2 [36, 0, 98, 92, 99]
40 0 [56, 59, 98, 0, 17]
43 0 [75, 65, 34, 42, 39]
43 1 [62, 0, 95, 75, 48]
46 0 [97, 75, 0, 13, 20]
46 1 [92, 0, 65, 36, 60]
47 0 [38, 11, 5, 98, 42]
51 0 [45, 1, 52, 46, 80]
52 0 [7, 5, 48, 63, 28]
52 1 [63, 98, 69, 48, 1]
52 2 [48, 45, 1, 74, 98]
52 3 [45, 48, 98, 33, 2]
55 0 [27, 45, 1, 80, 74]
55 2 [45, 1, 27, 37, 78]
56 0 [55, 7, 43, 1, 63]
57 2 [72, 45, 1, 49, 74]
57 3 [45, 49, 72, 99, 1]
59 0 [64, 71, 2, 69, 72]
59 1 [2, 76, 69, 64, 49]
59 2 [40, 64, 2, 69, 72]
59 3 [64, 69, 40, 2, 6]
60 0

C:\Users\ajk\AppData\Local\Continuum\miniconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: `item` has been deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


In [92]:
df.loc[df['sample']==57]

,sample,exit,prediction,scores,target,time,time_tx,correct,index_top5
228,57,0,"[1, 45, 72, 55, 99]","[0.47940167784690857, 0.3718308210372925, 0.10...",1,207.3101,6.2552,True,0
229,57,1,"[1, 45, 72, 63, 94]","[0.9434945583343506, 0.05228209123015404, 0.00...",1,316.6097,6.2552,True,0
230,57,2,"[72, 45, 1, 49, 74]","[0.6890237927436829, 0.16498732566833496, 0.09...",1,391.1214,6.2552,False,2
231,57,3,"[45, 49, 72, 99, 1]","[0.9252182841300964, 0.027236683294177055, 0.0...",1,415.4647,6.2552,False,4


In [93]:
df.groupby('exit').mean()

,sample,target,time,time_tx,correct,index_top5
exit,,,,,,
0,49.5,0.5,215.115127,5.631074,0.60,0.43
1,49.5,0.5,329.061245,5.631074,0.75,0.27
2,49.5,0.5,402.343290,5.631074,0.80,0.26
3,49.5,0.5,429.252994,5.631074,0.88,0.10


In [169]:
exit_best = np.zeros(4)
for i, d in df.groupby('sample'):
    #tmp = d[np.argmax(d.scores.tolist()[0]):1]
    exit = np.argmax(d.scores.tolist())//4
    # (d[exit:exit+1].exit.item())
    exit_best[exit] +=1


In [172]:
exit_best

array([ 0.,  5., 33., 62.])

In [171]:
exit_stats = []
correct = np.zeros(len(data[0]))
for sample in data:
    for i, exit in enumerate(sample):
        correct[i] += exit['correct']
    
    
print(correct/len(data))

KeyError: 0

In [28]:
works_data = json_normalize(data=data)
works_data.head(3)

AttributeError: 'list' object has no attribute 'values'